In [1]:
%load_ext lab_black
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import json

import os
from os.path import join

from t5_model_support_functions import (
    load_csv_files,
    get_vega_zero_table,
    get_columns_used_in_vz_query,
    nvBenchDatabase,
)

pd.options.mode.chained_assignment = None  # default='warn'

In [3]:
PROJECT_DIRECTORY = "/user/w266/w266-project-carlos"
NVBENCH_DIRECTORY = join(PROJECT_DIRECTORY, "ref_repos/nvBench/database")
NCNET_DATA_DIRECTORY = join(PROJECT_DIRECTORY, "ref_repos/ncNet/dataset/dataset_final")
SPIDER_DATABASE_DIRECTORY = join(PROJECT_DIRECTORY, "ref_repos/spider")

## nvBench

In [4]:
# nvBench
def json_to_dict(file_path):
    # Open the JSON file
    with open(file_path, "r") as file:
        # Parse the JSON data directly from the file object
        my_dict = json.load(file)

    return my_dict

In [5]:
nvBench = json_to_dict(join(PROJECT_DIRECTORY, "ref_repos/nvBench/NVBench.json"))

print(f"Sample of main nvBench keys {list(nvBench)[0:10]}\n\n")

print(f"First 10 records")
for i, k in enumerate(nvBench.keys()):
    json_data = nvBench[k]

    print(f"RECORD: {i}")
    print(json.dumps(json_data, indent=2))

    print("*" * 100)

    if i == 10:
        break

Sample of main nvBench keys ['3', '4', '5', '6', '7', '8', '9', '11', '12', '14']


First 10 records
RECORD: 0
{
  "vis_query": {
    "vis_part": "Visualize SCATTER",
    "data_part": {
      "sql_part": "SELECT FacID , count(*) FROM Faculty AS T1 JOIN Student AS T2 ON T1.FacID = T2.advisor GROUP BY T1.FacID",
      "binning": ""
    },
    "VQL": "Visualize SCATTER SELECT FacID , count(*) FROM Faculty AS T1 JOIN Student AS T2 ON T1.FacID = T2.advisor GROUP BY T1.FacID"
  },
  "chart": "Scatter",
  "hardness": "Medium",
  "db_id": "activity_1",
  "vis_obj": {
    "chart": "scatter",
    "x_name": "FacID",
    "y_name": "count(*)",
    "x_data": [
      [
        1121,
        1148,
        2192,
        2311,
        5718,
        7134,
        7271,
        7712,
        7723,
        7792,
        8423,
        8721,
        8722,
        8723,
        8741,
        8772,
        8918,
        9172
      ]
    ],
    "y_data": [
      [
        3,
        3,
        4,
        3,
   

Read nvBench as a dataframe

In [6]:
def read_nvBench(path_to_json):
    nvBench_json = json_to_dict(path_to_json)

    vql = []
    has_join = []
    sql = []
    sql_binning = []
    chart = []
    hardness = []
    db_id = []
    x_name = []
    y_name = []
    nl_queries = []
    number_of_queries = []

    for i, k in enumerate(nvBench_json.keys()):
        json_data = nvBench_json[k]

        # vis_query
        query = json_data["vis_query"]["VQL"]

        vql.append(query)
        has_join.append(True if "JOIN" in query else False)

        # sql_query
        sql.append(json_data["vis_query"]["data_part"]["sql_part"])
        sql_binning.append(json_data["vis_query"]["data_part"]["binning"])

        # other sections
        chart.append(json_data["chart"])
        hardness.append(json_data["hardness"])
        db_id.append(json_data["db_id"])
        x_name.append(json_data["vis_obj"]["x_name"])
        y_name.append(json_data["vis_obj"]["y_name"])

        nl_q = json_data["nl_queries"]
        nl_queries.append(nl_q)
        number_of_queries.append(len(nl_q))

    df = pd.DataFrame()
    df["vql"] = vql
    df["join_tables"] = has_join
    df["sql"] = sql
    df["sql_binning"] = sql_binning
    df["chart"] = chart
    df["hardness"] = hardness
    df["db_id"] = db_id
    df["x_name"] = x_name
    df["y_name"] = y_name
    df["nl_queries"] = nl_queries
    df["number_of_queries"] = number_of_queries

    return df


df_nvBench = read_nvBench(join(PROJECT_DIRECTORY, "ref_repos/nvBench/NVBench.json"))

df_nvBench["vql_len"] = df_nvBench["vql"].apply(lambda var: len(var.split()))

df_nvBench

,vql,join_tables,sql,sql_binning,chart,hardness,db_id,x_name,y_name,nl_queries,number_of_queries,vql_len
0,"Visualize SCATTER SELECT FacID , count(*) FROM...",True,"SELECT FacID , count(*) FROM Faculty AS T1 JOI...",,Scatter,Medium,activity_1,FacID,count(*),"[Show the faculty id of each faculty member, a...",5,21
1,"Visualize PIE SELECT Rank , count(*) FROM Facu...",True,"SELECT Rank , count(*) FROM Faculty AS T1 JOIN...",,Pie,Medium,activity_1,Rank,count(*),[Show all the faculty ranks and the number of ...,6,21
2,"Visualize BAR SELECT Rank , count(*) FROM Facu...",True,"SELECT Rank , count(*) FROM Faculty AS T1 JOIN...",,Bar,Medium,activity_1,Rank,count(*),[Show all the faculty ranks and the number of ...,6,21
3,"Visualize PIE SELECT Building , count(*) FROM ...",False,"SELECT Building , count(*) FROM Faculty GROUP ...",,Pie,Easy,activity_1,Building,count(*),[Show all the buildings along with the number ...,7,11
4,"Visualize BAR SELECT Building , count(*) FROM ...",False,"SELECT Building , count(*) FROM Faculty GROUP ...",,Bar,Easy,activity_1,Building,count(*),[Show all the buildings along with the number ...,7,11
...,...,...,...,...,...,...,...,...,...,...,...,...
7242,"Visualize BAR SELECT name , count(*) FROM user...",True,"SELECT name , count(*) FROM user_profiles AS T...",,Bar,Hard,twitter_1,name,count(*),[Find the name of each user and number of twee...,1,25
7243,"Visualize BAR SELECT Denomination , COUNT(*) F...",False,"SELECT Denomination , COUNT(*) FROM school GRO...",,Bar,Medium,school_player,Denomination,COUNT(*),[Create a bar chart showing the total number a...,2,15
7244,"Visualize BAR SELECT Denomination , COUNT(*) F...",False,"SELECT Denomination , COUNT(*) FROM school GRO...",,Bar,Medium,school_player,Denomination,COUNT(*),"[For each denomination, return the denominatio...",2,15
7245,"Visualize BAR SELECT Denomination , COUNT(*) F...",False,"SELECT Denomination , COUNT(*) FROM school GRO...",,Bar,Medium,school_player,Denomination,COUNT(*),"[For each denomination, return the denominatio...",2,15


Subset of nvBench without the joins (nj for "no join")

In [7]:
df_nvBench_nj = df_nvBench.loc[df_nvBench["join_tables"] == False]
print(f"number of vis queries: {len(df_nvBench_nj)}")
display(df_nvBench_nj[["hardness", "vql"]].groupby(by="hardness").count())

print(f"number of nl queries: {df_nvBench_nj['number_of_queries'].values.sum()}")

display(df_nvBench_nj[["hardness", "number_of_queries"]].groupby(by="hardness").sum())

number of vis queries: 4540


,vql
hardness,
Easy,1347
Extra Hard,374
Hard,731
Medium,2088


number of nl queries: 18067


,number_of_queries
hardness,
Easy,5730
Extra Hard,1628
Hard,2585
Medium,8124


In [8]:
df_nvBench_nj_exp = (
    df_nvBench_nj.explode("nl_queries")
    .reset_index(drop=True)
    .drop(columns="number_of_queries")
)
df_nvBench_nj_exp

,vql,join_tables,sql,sql_binning,chart,hardness,db_id,x_name,y_name,nl_queries,vql_len
0,"Visualize PIE SELECT Building , count(*) FROM ...",False,"SELECT Building , count(*) FROM Faculty GROUP ...",,Pie,Easy,activity_1,Building,count(*),Show all the buildings along with the number o...,11
1,"Visualize PIE SELECT Building , count(*) FROM ...",False,"SELECT Building , count(*) FROM Faculty GROUP ...",,Pie,Easy,activity_1,Building,count(*),Show all the buildings along with the number o...,11
2,"Visualize PIE SELECT Building , count(*) FROM ...",False,"SELECT Building , count(*) FROM Faculty GROUP ...",,Pie,Easy,activity_1,Building,count(*),Return a pie on how many faculty members does ...,11
3,"Visualize PIE SELECT Building , count(*) FROM ...",False,"SELECT Building , count(*) FROM Faculty GROUP ...",,Pie,Easy,activity_1,Building,count(*),Draw a pie chart for how many faculty members ...,11
4,"Visualize PIE SELECT Building , count(*) FROM ...",False,"SELECT Building , count(*) FROM Faculty GROUP ...",,Pie,Easy,activity_1,Building,count(*),How many faculty members does each building ha...,11
...,...,...,...,...,...,...,...,...,...,...,...
18062,"Visualize BAR SELECT Denomination , COUNT(*) F...",False,"SELECT Denomination , COUNT(*) FROM school GRO...",,Bar,Medium,school_player,Denomination,COUNT(*),Create a bar chart showing the total number ac...,15
18063,"Visualize BAR SELECT Denomination , COUNT(*) F...",False,"SELECT Denomination , COUNT(*) FROM school GRO...",,Bar,Medium,school_player,Denomination,COUNT(*),"For each denomination, return the denomination...",15
18064,"Visualize BAR SELECT Denomination , COUNT(*) F...",False,"SELECT Denomination , COUNT(*) FROM school GRO...",,Bar,Medium,school_player,Denomination,COUNT(*),Create a bar chart showing the total number ac...,15
18065,"Visualize BAR SELECT Denomination , COUNT(*) F...",False,"SELECT Denomination , COUNT(*) FROM school GRO...",,Bar,Medium,school_player,Denomination,COUNT(*),"For each denomination, return the denomination...",15


In [9]:
# df_nvBench_nj_exp.to_csv("nvBench_nj.csv")


df_nvBench_nj_exp["vql_len"].values.max()

32

## NCNET cross check

In [10]:
df_ncNet = load_csv_files(
    csv_paths=[
        join(NCNET_DATA_DIRECTORY, "train.csv"),
        join(NCNET_DATA_DIRECTORY, "dev.csv"),
        join(NCNET_DATA_DIRECTORY, "test.csv"),
    ],
    focus_columns=[
        "db_id",
        "chart",
        "hardness",
        "query",
        "question",
    ],
    drop_duplicates=True,
    dropna=False,
    shuffle=False,
    single_output=True,
)

df_ncNet.head(2)

Loading 'train.csv'
Number of records in /user/w266/w266-project-carlos/ref_repos/ncNet/dataset/dataset_final/train.csv: 25238

Loading 'dev.csv'
Number of records in /user/w266/w266-project-carlos/ref_repos/ncNet/dataset/dataset_final/dev.csv: 1430
-> Merged!!

Loading 'test.csv'
Number of records in /user/w266/w266-project-carlos/ref_repos/ncNet/dataset/dataset_final/test.csv: 4920
-> Merged!!

Focusing on the following columns: ['db_id', 'chart', 'hardness', 'query', 'question']

Searching for duplicate rows in focus columns...
A total of 15785 records were loaded (15803 records dropped after duplicate filter)

returning a single file...


,tvBench_id,db_id,chart,hardness,query,question,vega_zero,mentioned_columns,mentioned_values,query_template,source,labels,token_types
0,1000@y_name@DESC,customers_and_products_contacts,Bar,Medium,"Visualize BAR SELECT product_name , COUNT(prod...",Bar chart x axis product name y axis how many ...,mark bar data products encoding x product_name...,product_name,Sony,mark [T] data products encoding x [X] y aggreg...,<N> Bar chart x axis product name y axis how m...,mark bar data products encoding x product_name...,nl nl nl nl nl nl nl nl nl nl nl nl nl nl nl n...
2,2463@x_name@ASC,network_2,Bar,Easy,"Visualize BAR SELECT job , min(age) FROM Perso...",how old is the youngest person for each job ? ...,mark bar data person encoding x job y aggregat...,job age name,Zach Bob Dan,mark [T] data person encoding x [X] y aggregat...,<N> how old is the youngest person for each jo...,mark bar data person encoding x job y aggregat...,nl nl nl nl nl nl nl nl nl nl nl nl nl nl nl n...


Entries in the ncNet dataset that are in nvBench

In [11]:
print(df_ncNet.columns)

Index(['tvBench_id', 'db_id', 'chart', 'hardness', 'query', 'question',
       'vega_zero', 'mentioned_columns', 'mentioned_values', 'query_template',
       'source', 'labels', 'token_types'],
      dtype='object')


### Find how many question, query pairs are common

In [12]:
import re


def replace_at_index(string, char, index):
    index = len(string) - 1 if index < 0 else index
    return string[:index] + char + string[index + 1 :]


def sperarate_character_by_spaces(sentence, character, process_char_mid_word=False):
    regex_meta_characters = ["^", "$", "*", "+", "?", ".", "(", ")"]

    regex_char = ("\\" + character) if character in regex_meta_characters else character

    sentence_list = []

    for w in sentence.split():
        if character not in w:
            sentence_list.append(w)
        elif w == character:
            sentence_list.append(w)
        else:
            # Starting pos replace
            if w.startswith(character):
                w = replace_at_index(w, character + " ", 0)

            # End pos replace
            if w.endswith(character):
                w = replace_at_index(w, " " + character, -1)

            # Mid pos replace
            if process_char_mid_word:
                pattern = r"(\w)" + regex_char + r"(\w)"
                repl = r" " + character + r" "
                repl_pattern = r"\1" + repl + r"\2"
                w = re.sub(pattern=pattern, repl=repl_pattern, string=w)

            sentence_list.append(w)

    return " ".join(sentence_list)


sentence = "(Visuali.ze. LINE SELECT date_address_to , AVG(monthly_rental) FROM Stud.ent_Addresses GRO!UP BY other_details , date_address_to ORDER BY monthly_rental DESC"
print(sentence)
print(
    sperarate_character_by_spaces(
        sentence=sentence, character="!", process_char_mid_word=True
    )
)

(Visuali.ze. LINE SELECT date_address_to , AVG(monthly_rental) FROM Stud.ent_Addresses GRO!UP BY other_details , date_address_to ORDER BY monthly_rental DESC
(Visuali.ze. LINE SELECT date_address_to , AVG(monthly_rental) FROM Stud.ent_Addresses GRO ! UP BY other_details , date_address_to ORDER BY monthly_rental DESC


#### Create 2 dataframes with the same column names to compare how many lines are repeated

- there was some formatting done to the ncNet databse which were replicated on the nvBench one

In [13]:
df1 = df_ncNet[["query", "question", "db_id", "chart"]]
# df1["question"] = df1["question"].str.lower()
# df1["query"] = df1["query"].str.lower()

df1["source"] = ["ncNet"] * len(df1)
df1.sort_values(by=["db_id", "chart", "query", "question"], inplace=True)


df2 = df_nvBench_nj_exp[["vql", "nl_queries", "db_id", "chart"]]
df2.columns = ["query", "question", "db_id", "chart"]
df2["question"] = df2["question"].apply(sperarate_character_by_spaces, args=("."))
df2["question"] = df2["question"].apply(sperarate_character_by_spaces, args=(","))
df2["question"] = df2["question"].apply(sperarate_character_by_spaces, args=("?"))
# df2["question"] = df2["question"].str.lower()
# df2["query"] = df2["query"].str.lower()

df2["source"] = ["nvBench"] * len(df2)
df2.sort_values(by=["db_id", "chart", "query", "question"], inplace=True)

In [14]:
print(f"df1(ncNet) size: {len(df1)}")
print(f"df2(nvBench) size: {len(df2)}")

df_dup = pd.concat([df1, df2])
print(f"df_dup(joined) size: {len(df_dup)}")

df_dup.drop_duplicates(
    subset=["question", "query", "db_id", "chart"], inplace=True, keep="last"
)

print(f"df_dup size: {len(df_dup)} -> after filter")

df1(ncNet) size: 15785
df2(nvBench) size: 18067
df_dup(joined) size: 33852
df_dup size: 18125 -> after filter


In [15]:
df_dup.groupby(by="source").count()

,query,question,db_id,chart
source,,,,
ncNet,67,67,67,67
nvBench,18058,18058,18058,18058


### ncNet check tables used and how many of the mentioned columns are used in the query

In [16]:
df_ncNet = load_csv_files(
    csv_paths=[
        join(NCNET_DATA_DIRECTORY, "train.csv"),
        join(NCNET_DATA_DIRECTORY, "dev.csv"),
        join(NCNET_DATA_DIRECTORY, "test.csv"),
    ],
    focus_columns=[
        "db_id",
        "chart",
        "hardness",
        "query",
        "question",
    ],
    drop_duplicates=True,
    dropna=False,
    shuffle=False,
    single_output=True,
)

df_ncNet["mentioned_columns"] = df_ncNet["mentioned_columns"].apply(
    lambda var: [] if pd.isna(var) else var.split()
)

df_ncNet["table"] = df_ncNet.apply(get_vega_zero_table, axis=1)

print(df_ncNet.shape)
df_ncNet

Loading 'train.csv'
Number of records in /user/w266/w266-project-carlos/ref_repos/ncNet/dataset/dataset_final/train.csv: 25238

Loading 'dev.csv'
Number of records in /user/w266/w266-project-carlos/ref_repos/ncNet/dataset/dataset_final/dev.csv: 1430
-> Merged!!

Loading 'test.csv'
Number of records in /user/w266/w266-project-carlos/ref_repos/ncNet/dataset/dataset_final/test.csv: 4920
-> Merged!!

Focusing on the following columns: ['db_id', 'chart', 'hardness', 'query', 'question']

Searching for duplicate rows in focus columns...
A total of 15785 records were loaded (15803 records dropped after duplicate filter)

returning a single file...
(15785, 14)


,tvBench_id,db_id,chart,hardness,query,question,vega_zero,mentioned_columns,mentioned_values,query_template,source,labels,token_types,table
0,1000@y_name@DESC,customers_and_products_contacts,Bar,Medium,"Visualize BAR SELECT product_name , COUNT(prod...",Bar chart x axis product name y axis how many ...,mark bar data products encoding x product_name...,[product_name],Sony,mark [T] data products encoding x [X] y aggreg...,<N> Bar chart x axis product name y axis how m...,mark bar data products encoding x product_name...,nl nl nl nl nl nl nl nl nl nl nl nl nl nl nl n...,products
2,2463@x_name@ASC,network_2,Bar,Easy,"Visualize BAR SELECT job , min(age) FROM Perso...",how old is the youngest person for each job ? ...,mark bar data person encoding x job y aggregat...,"[job, age, name]",Zach Bob Dan,mark [T] data person encoding x [X] y aggregat...,<N> how old is the youngest person for each jo...,mark bar data person encoding x job y aggregat...,nl nl nl nl nl nl nl nl nl nl nl nl nl nl nl n...,person
4,2545@y_name@DESC,pets_1,Bar,Medium,"Visualize BAR SELECT PetType , avg(pet_age) FR...",Please give me a bar chart to show the average...,mark bar data pets encoding x pettype y aggreg...,"[pet_age, petid, pettype]",cat,mark [T] data pets encoding x [X] y aggregate ...,<N> Please give me a bar chart to show the ave...,mark bar data pets encoding x pettype y aggreg...,nl nl nl nl nl nl nl nl nl nl nl nl nl nl nl n...,pets
6,2615@y_name@ASC,products_for_hire,Bar,Extra Hard,"Visualize BAR SELECT payment_date , COUNT(paym...",What are the payment date of the payment with ...,mark bar data payments encoding x payment_date...,"[payment_date, amount_paid, payment_type_code,...",Check 0,mark [T] data payments encoding x [X] y aggreg...,<N> What are the payment date of the payment w...,mark bar data payments encoding x payment_date...,nl nl nl nl nl nl nl nl nl nl nl nl nl nl nl n...,payments
8,1304,election,Bar,Easy,"Visualize BAR SELECT County_name , Population ...",What are the name and population of each count...,mark bar data county encoding x county_name y ...,[population],NaN,mark [T] data county encoding x [X] y aggregat...,<N> What are the name and population of each c...,mark bar data county encoding x county_name y ...,nl nl nl nl nl nl nl nl nl nl nl nl nl nl nl n...,county
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31578,79@y_name@ASC,apartment_rentals,Bar,Hard,"Visualize BAR SELECT date_of_birth , COUNT(dat...",What are the number of dates of birth of all t...,mark bar data guests encoding x date_of_birth ...,"[gender_code, guest_first_name, guest_last_name]",Male Gabe Toy,mark [T] data guests encoding x [X] y aggregat...,<N> What are the number of dates of birth of a...,mark bar data guests encoding x date_of_birth ...,nl nl nl nl nl nl nl nl nl nl nl nl nl nl nl n...,guests
31580,2914@x_name@DESC,swimming,Bar,Easy,"Visualize BAR SELECT name , ID FROM swimmer OR...","Plot id by grouped by name as a bar graph , sh...",mark bar data swimmer encoding x name y aggreg...,"[id, name, time]",NaN,mark [T] data swimmer encoding x [X] y aggrega...,<N> Plot id by grouped by name as a bar graph ...,mark bar data swimmer encoding x name y aggreg...,nl nl nl nl nl nl nl nl nl nl nl nl nl nl nl n...,swimmer
31582,2763@x_name@ASC,ship_mission,Bar,Easy,"Visualize BAR SELECT Fate , COUNT(Fate) FROM m...",Compare the total number of each fate with a b...,mark bar data mission encoding x fate y aggreg...,[fate],NaN,mark [T] data mission encoding x [X] y aggrega...,<N> Compare the total number of each fate with...,mark bar data mission encoding x fate y aggreg...,nl nl nl nl nl nl nl nl nl nl nl nl nl nl nl n...,mission
31584,2739@y_name@ASC,ship_1,Bar,Medium,"Visualize BAR SELECT Flag , count(*) FROM ship...","Show the total number from each flag , and sor...",mark bar data ship encoding x flag y aggregate...,"[flag, type]",NaN,mark [T] data ship encoding x [X] y aggregate ...,"<N> Show the total number from each flag , and...",mar

### Check if all the columns and tables are found on the nvBench Database

In [17]:
# Load the nvBench database info
nvBench_Database = nvBenchDatabase(path=NVBENCH_DIRECTORY)
df_nvBench_db = nvBench_Database.get_database_table_details(lower_case=True)
df_nvBench_db = df_nvBench_db[df_nvBench_db.columns.drop("successful_open")]
df_nvBench_db.columns = ["db_id", "table", "nvBench_column_names"]
df_nvBench_db

,db_id,table,nvBench_column_names
0,academic,author,"[aid, homepage, name, oid]"
1,academic,conference,"[cid, homepage, name]"
2,academic,domain,"[did, name]"
3,academic,domain_author,"[aid, did]"
4,academic,domain_conference,"[cid, did]"
...,...,...,...
871,yelp,user,"[uid, user_id, name]"
872,yelp,checkin,"[cid, business_id, count, day]"
873,yelp,neighbourhood,"[id, business_id, neighbourhood_name]"
874,yelp,review,"[rid, business_id, user_id, rating, text, year..."


In [18]:
# Merging with ncNet
df_ncNet = pd.merge(df_ncNet, df_nvBench_db, on=["db_id", "table"], how="left")

df_ncNet["mentioned_columns_found_in_db"] = df_ncNet.apply(
    lambda row: set(row["mentioned_columns"]).issubset(row["nvBench_column_names"]),
    axis=1,
)

df_ncNet

,tvBench_id,db_id,chart,hardness,query,question,vega_zero,mentioned_columns,mentioned_values,query_template,source,labels,token_types,table,nvBench_column_names,mentioned_columns_found_in_db
0,1000@y_name@DESC,customers_and_products_contacts,Bar,Medium,"Visualize BAR SELECT product_name , COUNT(prod...",Bar chart x axis product name y axis how many ...,mark bar data products encoding x product_name...,[product_name],Sony,mark [T] data products encoding x [X] y aggreg...,<N> Bar chart x axis product name y axis how m...,mark bar data products encoding x product_name...,nl nl nl nl nl nl nl nl nl nl nl nl nl nl nl n...,products,"[product_id, product_type_code, product_name, ...",True
1,2463@x_name@ASC,network_2,Bar,Easy,"Visualize BAR SELECT job , min(age) FROM Perso...",how old is the youngest person for each job ? ...,mark bar data person encoding x job y aggregat...,"[job, age, name]",Zach Bob Dan,mark [T] data person encoding x [X] y aggregat...,<N> how old is the youngest person for each jo...,mark bar data person encoding x job y aggregat...,nl nl nl nl nl nl nl nl nl nl nl nl nl nl nl n...,person,"[name, age, city, gender, job]",True
2,2545@y_name@DESC,pets_1,Bar,Medium,"Visualize BAR SELECT PetType , avg(pet_age) FR...",Please give me a bar chart to show the average...,mark bar data pets encoding x pettype y aggreg...,"[pet_age, petid, pettype]",cat,mark [T] data pets encoding x [X] y aggregate ...,<N> Please give me a bar chart to show the ave...,mark bar data pets encoding x pettype y aggreg...,nl nl nl nl nl nl nl nl nl nl nl nl nl nl nl n...,pets,"[petid, pettype, pet_age, weight]",True
3,2615@y_name@ASC,products_for_hire,Bar,Extra Hard,"Visualize BAR SELECT payment_date , COUNT(paym...",What are the payment date of the payment with ...,mark bar data payments encoding x payment_date...,"[payment_date, amount_paid, payment_type_code,...",Check 0,mark [T] data payments encoding x [X] y aggreg...,<N> What are the payment date of the payment w...,mark bar data payments encoding x payment_date...,nl nl nl nl nl nl nl nl nl nl nl nl nl nl nl n...,payments,"[payment_id, booking_id, customer_id, payment_...",True
4,1304,election,Bar,Easy,"Visualize BAR SELECT County_name , Population ...",What are the name and population of each count...,mark bar data county encoding x county_name y ...,[population],NaN,mark [T] data county encoding x [X] y aggregat...,<N> What are the name and population of each c...,mark bar data county encoding x county_name y ...,nl nl nl nl nl nl nl nl nl nl nl nl nl nl nl n...,county,"[county_id, county_name, population, zip_code]",True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15780,79@y_name@ASC,apartment_rentals,Bar,Hard,"Visualize BAR SELECT date_of_birth , COUNT(dat...",What are the number of dates of birth of all t...,mark bar data guests encoding x date_of_birth ...,"[gender_code, guest_first_name, guest_last_name]",Male Gabe Toy,mark [T] data guests encoding x [X] y aggregat...,<N> What are the number of dates of birth of a...,mark bar data guests encoding x date_of_birth ...,nl nl nl nl nl nl nl nl nl nl nl nl nl nl nl n...,guests,"[guest_id, gender_code, guest_first_name, gues...",True
15781,2914@x_name@DESC,swimming,Bar,Easy,"Visualize BAR SELECT name , ID FROM swimmer OR...","Plot id by grouped by name as a bar graph , sh...",mark bar data swimmer encoding x name y aggreg...,"[id, name, time]",NaN,mark [T] data swimmer encoding x [X] y aggrega...,<N> Plot id by grouped by name as a bar graph ...,mark bar data swimmer encoding x name y aggreg...,nl nl nl nl nl nl nl nl nl nl nl nl nl nl nl n...,swimmer,"[id, name, nationality, meter_100, meter_200, ...",True
15782,2763@x_name@ASC,ship_mission,Bar,Easy,"Visualize BAR SELECT Fate , COUNT(Fate) FROM m...",Compare the total number of each fate with a b...,mark bar data mission encoding x fate y aggreg...,[fate],NaN,mark [T] data mission encoding x [X] y aggrega...,<N> Compare the total number of each fate with...,mark b

In [19]:
# Checking if all the db_ids in ncNet appear on nvBench
set(df_ncNet["db_id"].values).issubset(set(df_nvBench_db["db_id"].values))

True

In [20]:
# checking if there are any columns where the ncNet table name is not found on nvBench
len(df_ncNet.loc[df_ncNet["nvBench_column_names"].isna()])

0

In [21]:
# Checking if all the columns are found in nvBench db
len(df_ncNet.loc[df_ncNet["mentioned_columns_found_in_db"] == False])

0

In [22]:
# Modifying columns used to pull from nvBench since not al mentioned_culumns have values
df_ncNet["columns_used"] = df_ncNet.apply(
    get_columns_used_in_vz_query, axis=1, args=(["nvBench_column_names"])
)
df_ncNet

,tvBench_id,db_id,chart,hardness,query,question,vega_zero,mentioned_columns,mentioned_values,query_template,source,labels,token_types,table,nvBench_column_names,mentioned_columns_found_in_db,columns_used
0,1000@y_name@DESC,customers_and_products_contacts,Bar,Medium,"Visualize BAR SELECT product_name , COUNT(prod...",Bar chart x axis product name y axis how many ...,mark bar data products encoding x product_name...,[product_name],Sony,mark [T] data products encoding x [X] y aggreg...,<N> Bar chart x axis product name y axis how m...,mark bar data products encoding x product_name...,nl nl nl nl nl nl nl nl nl nl nl nl nl nl nl n...,products,"[product_id, product_type_code, product_name, ...",True,[product_name]
1,2463@x_name@ASC,network_2,Bar,Easy,"Visualize BAR SELECT job , min(age) FROM Perso...",how old is the youngest person for each job ? ...,mark bar data person encoding x job y aggregat...,"[job, age, name]",Zach Bob Dan,mark [T] data person encoding x [X] y aggregat...,<N> how old is the youngest person for each jo...,mark bar data person encoding x job y aggregat...,nl nl nl nl nl nl nl nl nl nl nl nl nl nl nl n...,person,"[name, age, city, gender, job]",True,"[age, job]"
2,2545@y_name@DESC,pets_1,Bar,Medium,"Visualize BAR SELECT PetType , avg(pet_age) FR...",Please give me a bar chart to show the average...,mark bar data pets encoding x pettype y aggreg...,"[pet_age, petid, pettype]",cat,mark [T] data pets encoding x [X] y aggregate ...,<N> Please give me a bar chart to show the ave...,mark bar data pets encoding x pettype y aggreg...,nl nl nl nl nl nl nl nl nl nl nl nl nl nl nl n...,pets,"[petid, pettype, pet_age, weight]",True,"[pettype, pet_age]"
3,2615@y_name@ASC,products_for_hire,Bar,Extra Hard,"Visualize BAR SELECT payment_date , COUNT(paym...",What are the payment date of the payment with ...,mark bar data payments encoding x payment_date...,"[payment_date, amount_paid, payment_type_code,...",Check 0,mark [T] data payments encoding x [X] y aggreg...,<N> What are the payment date of the payment w...,mark bar data payments encoding x payment_date...,nl nl nl nl nl nl nl nl nl nl nl nl nl nl nl n...,payments,"[payment_id, booking_id, customer_id, payment_...",True,"[payment_type_code, payment_date, amount_paid]"
4,1304,election,Bar,Easy,"Visualize BAR SELECT County_name , Population ...",What are the name and population of each count...,mark bar data county encoding x county_name y ...,[population],NaN,mark [T] data county encoding x [X] y aggregat...,<N> What are the name and population of each c...,mark bar data county encoding x county_name y ...,nl nl nl nl nl nl nl nl nl nl nl nl nl nl nl n...,county,"[county_id, county_name, population, zip_code]",True,"[county_name, population]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15780,79@y_name@ASC,apartment_rentals,Bar,Hard,"Visualize BAR SELECT date_of_birth , COUNT(dat...",What are the number of dates of birth of all t...,mark bar data guests encoding x date_of_birth ...,"[gender_code, guest_first_name, guest_last_name]",Male Gabe Toy,mark [T] data guests encoding x [X] y aggregat...,<N> What are the number of dates of birth of a...,mark bar data guests encoding x date_of_birth ...,nl nl nl nl nl nl nl nl nl nl nl nl nl nl nl n...,guests,"[guest_id, gender_code, guest_first_name, gues...",True,"[gender_code, date_of_birth]"
15781,2914@x_name@DESC,swimming,Bar,Easy,"Visualize BAR SELECT name , ID FROM swimmer OR...","Plot id by grouped by name as a bar graph , sh...",mark bar data swimmer encoding x name y aggreg...,"[id, name, time]",NaN,mark [T] data swimmer encoding x [X] y aggrega...,<N> Plot id by grouped by name as a bar graph ...,mark bar data swimmer encoding x name y aggreg...,nl nl nl nl nl nl nl nl nl nl nl nl nl nl nl n...,swimmer,"[id, name, nationality, meter_100, meter_200, ...",True,"[id, name]"
15782,2763@x_name@ASC,ship_mission,Bar,Easy,"Visualize BAR SELECT Fate , COUNT(Fate) FROM m...",Compare the total number of ea

#### Redoing the above in a fucntion

In [23]:
def attach_nvBench_info(df_data: pd.DataFrame, nvBench_db_path: str) -> pd.DataFrame:
    df_data["mentioned_columns"] = df_data["mentioned_columns"].apply(
        lambda var: [] if pd.isna(var) else var.split()
    )

    df_data["table"] = df_data.apply(get_vega_zero_table, axis=1)

    # Load the nvBench database info
    nvBench_Database = nvBenchDatabase(path=nvBench_db_path)
    df_nvBench_db = nvBench_Database.get_database_table_details(lower_case=True)
    df_nvBench_db = df_nvBench_db[df_nvBench_db.columns.drop("successful_open")]
    df_nvBench_db.columns = ["db_id", "table", "nvBench_column_names"]

    # Checking if all the db_ids in ncNet appear on nvBench
    assert set(df_data["db_id"].values).issubset(set(df_nvBench_db["db_id"].values))

    # Merging with ncNet
    df_data = pd.merge(df_data, df_nvBench_db, on=["db_id", "table"], how="left")

    df_data["mentioned_columns_found_in_db"] = df_data.apply(
        lambda row: set(row["mentioned_columns"]).issubset(row["nvBench_column_names"]),
        axis=1,
    )

    # checking if there are any columns where the ncNet table name is not found on nvBench
    assert len(df_data.loc[df_data["nvBench_column_names"].isna()]) == 0

    # Checking if all the columns are found in nvBench db
    assert len(df_data.loc[df_data["mentioned_columns_found_in_db"] == False]) == 0

    # Get columns used in the vega-zero query
    df_data["columns_used"] = df_data.apply(
        get_columns_used_in_vz_query, axis=1, args=(["nvBench_column_names"])
    )

    return df_data[df_data.columns.drop("mentioned_columns_found_in_db")]


df_ncNet = load_csv_files(
    csv_paths=[
        join(NCNET_DATA_DIRECTORY, "train.csv"),
        join(NCNET_DATA_DIRECTORY, "dev.csv"),
        join(NCNET_DATA_DIRECTORY, "test.csv"),
    ],
    focus_columns=[
        "db_id",
        "chart",
        "hardness",
        "query",
        "question",
    ],
    drop_duplicates=True,
    dropna=False,
    shuffle=False,
    single_output=True,
)

df_ncNet = attach_nvBench_info(df_ncNet, NVBENCH_DIRECTORY)

df_ncNet

Loading 'train.csv'
Number of records in /user/w266/w266-project-carlos/ref_repos/ncNet/dataset/dataset_final/train.csv: 25238

Loading 'dev.csv'
Number of records in /user/w266/w266-project-carlos/ref_repos/ncNet/dataset/dataset_final/dev.csv: 1430
-> Merged!!

Loading 'test.csv'
Number of records in /user/w266/w266-project-carlos/ref_repos/ncNet/dataset/dataset_final/test.csv: 4920
-> Merged!!

Focusing on the following columns: ['db_id', 'chart', 'hardness', 'query', 'question']

Searching for duplicate rows in focus columns...
A total of 15785 records were loaded (15803 records dropped after duplicate filter)

returning a single file...


,tvBench_id,db_id,chart,hardness,query,question,vega_zero,mentioned_columns,mentioned_values,query_template,source,labels,token_types,table,nvBench_column_names,columns_used
0,1000@y_name@DESC,customers_and_products_contacts,Bar,Medium,"Visualize BAR SELECT product_name , COUNT(prod...",Bar chart x axis product name y axis how many ...,mark bar data products encoding x product_name...,[product_name],Sony,mark [T] data products encoding x [X] y aggreg...,<N> Bar chart x axis product name y axis how m...,mark bar data products encoding x product_name...,nl nl nl nl nl nl nl nl nl nl nl nl nl nl nl n...,products,"[product_id, product_type_code, product_name, ...",[product_name]
1,2463@x_name@ASC,network_2,Bar,Easy,"Visualize BAR SELECT job , min(age) FROM Perso...",how old is the youngest person for each job ? ...,mark bar data person encoding x job y aggregat...,"[job, age, name]",Zach Bob Dan,mark [T] data person encoding x [X] y aggregat...,<N> how old is the youngest person for each jo...,mark bar data person encoding x job y aggregat...,nl nl nl nl nl nl nl nl nl nl nl nl nl nl nl n...,person,"[name, age, city, gender, job]","[age, job]"
2,2545@y_name@DESC,pets_1,Bar,Medium,"Visualize BAR SELECT PetType , avg(pet_age) FR...",Please give me a bar chart to show the average...,mark bar data pets encoding x pettype y aggreg...,"[pet_age, petid, pettype]",cat,mark [T] data pets encoding x [X] y aggregate ...,<N> Please give me a bar chart to show the ave...,mark bar data pets encoding x pettype y aggreg...,nl nl nl nl nl nl nl nl nl nl nl nl nl nl nl n...,pets,"[petid, pettype, pet_age, weight]","[pettype, pet_age]"
3,2615@y_name@ASC,products_for_hire,Bar,Extra Hard,"Visualize BAR SELECT payment_date , COUNT(paym...",What are the payment date of the payment with ...,mark bar data payments encoding x payment_date...,"[payment_date, amount_paid, payment_type_code,...",Check 0,mark [T] data payments encoding x [X] y aggreg...,<N> What are the payment date of the payment w...,mark bar data payments encoding x payment_date...,nl nl nl nl nl nl nl nl nl nl nl nl nl nl nl n...,payments,"[payment_id, booking_id, customer_id, payment_...","[payment_type_code, payment_date, amount_paid]"
4,1304,election,Bar,Easy,"Visualize BAR SELECT County_name , Population ...",What are the name and population of each count...,mark bar data county encoding x county_name y ...,[population],NaN,mark [T] data county encoding x [X] y aggregat...,<N> What are the name and population of each c...,mark bar data county encoding x county_name y ...,nl nl nl nl nl nl nl nl nl nl nl nl nl nl nl n...,county,"[county_id, county_name, population, zip_code]","[county_name, population]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15780,79@y_name@ASC,apartment_rentals,Bar,Hard,"Visualize BAR SELECT date_of_birth , COUNT(dat...",What are the number of dates of birth of all t...,mark bar data guests encoding x date_of_birth ...,"[gender_code, guest_first_name, guest_last_name]",Male Gabe Toy,mark [T] data guests encoding x [X] y aggregat...,<N> What are the number of dates of birth of a...,mark bar data guests encoding x date_of_birth ...,nl nl nl nl nl nl nl nl nl nl nl nl nl nl nl n...,guests,"[guest_id, gender_code, guest_first_name, gues...","[gender_code, date_of_birth]"
15781,2914@x_name@DESC,swimming,Bar,Easy,"Visualize BAR SELECT name , ID FROM swimmer OR...","Plot id by grouped by name as a bar graph , sh...",mark bar data swimmer encoding x name y aggreg...,"[id, name, time]",NaN,mark [T] data swimmer encoding x [X] y aggrega...,<N> Plot id by grouped by name as a bar graph ...,mark bar data swimmer encoding x name y aggreg...,nl nl nl nl nl nl nl nl nl nl nl nl nl nl nl n...,swimmer,"[id, name, nationality, meter_100, meter_200, ...","[id, name]"
15782,2763@x_name@ASC,ship_mission,Bar,Easy,"Visualize BAR SELECT Fate , COUNT(Fate) FROM m...",Compare the total number of each fate with a b...,mark bar data mission encoding x fate y aggreg...

### Re-building the source with new columns

In [24]:
# Re-building the source with new columns
def build_vega_zero_source(row, ref_column="mentioned_columns", boost: int = 1):
    question = row["question"]
    template = row["query_template"]
    columns = " ".join(row[ref_column] * boost)
    table = row["table"]
    values = row["mentioned_values"]

    source = f"<N> {question} </N> <C> {template} </C> <D> {table} <COL> {columns} </COL> <VAL> {values} </VAL> </D>"
    return source


df_ncNet["source_new"] = df_ncNet.apply(
    build_vega_zero_source, axis=1, args=["columns_used", 2]
)

df_ncNet

,tvBench_id,db_id,chart,hardness,query,question,vega_zero,mentioned_columns,mentioned_values,query_template,source,labels,token_types,table,nvBench_column_names,columns_used,source_new
0,1000@y_name@DESC,customers_and_products_contacts,Bar,Medium,"Visualize BAR SELECT product_name , COUNT(prod...",Bar chart x axis product name y axis how many ...,mark bar data products encoding x product_name...,[product_name],Sony,mark [T] data products encoding x [X] y aggreg...,<N> Bar chart x axis product name y axis how m...,mark bar data products encoding x product_name...,nl nl nl nl nl nl nl nl nl nl nl nl nl nl nl n...,products,"[product_id, product_type_code, product_name, ...",[product_name],<N> Bar chart x axis product name y axis how m...
1,2463@x_name@ASC,network_2,Bar,Easy,"Visualize BAR SELECT job , min(age) FROM Perso...",how old is the youngest person for each job ? ...,mark bar data person encoding x job y aggregat...,"[job, age, name]",Zach Bob Dan,mark [T] data person encoding x [X] y aggregat...,<N> how old is the youngest person for each jo...,mark bar data person encoding x job y aggregat...,nl nl nl nl nl nl nl nl nl nl nl nl nl nl nl n...,person,"[name, age, city, gender, job]","[age, job]",<N> how old is the youngest person for each jo...
2,2545@y_name@DESC,pets_1,Bar,Medium,"Visualize BAR SELECT PetType , avg(pet_age) FR...",Please give me a bar chart to show the average...,mark bar data pets encoding x pettype y aggreg...,"[pet_age, petid, pettype]",cat,mark [T] data pets encoding x [X] y aggregate ...,<N> Please give me a bar chart to show the ave...,mark bar data pets encoding x pettype y aggreg...,nl nl nl nl nl nl nl nl nl nl nl nl nl nl nl n...,pets,"[petid, pettype, pet_age, weight]","[pettype, pet_age]",<N> Please give me a bar chart to show the ave...
3,2615@y_name@ASC,products_for_hire,Bar,Extra Hard,"Visualize BAR SELECT payment_date , COUNT(paym...",What are the payment date of the payment with ...,mark bar data payments encoding x payment_date...,"[payment_date, amount_paid, payment_type_code,...",Check 0,mark [T] data payments encoding x [X] y aggreg...,<N> What are the payment date of the payment w...,mark bar data payments encoding x payment_date...,nl nl nl nl nl nl nl nl nl nl nl nl nl nl nl n...,payments,"[payment_id, booking_id, customer_id, payment_...","[payment_type_code, payment_date, amount_paid]",<N> What are the payment date of the payment w...
4,1304,election,Bar,Easy,"Visualize BAR SELECT County_name , Population ...",What are the name and population of each count...,mark bar data county encoding x county_name y ...,[population],NaN,mark [T] data county encoding x [X] y aggregat...,<N> What are the name and population of each c...,mark bar data county encoding x county_name y ...,nl nl nl nl nl nl nl nl nl nl nl nl nl nl nl n...,county,"[county_id, county_name, population, zip_code]","[county_name, population]",<N> What are the name and population of each c...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15780,79@y_name@ASC,apartment_rentals,Bar,Hard,"Visualize BAR SELECT date_of_birth , COUNT(dat...",What are the number of dates of birth of all t...,mark bar data guests encoding x date_of_birth ...,"[gender_code, guest_first_name, guest_last_name]",Male Gabe Toy,mark [T] data guests encoding x [X] y aggregat...,<N> What are the number of dates of birth of a...,mark bar data guests encoding x date_of_birth ...,nl nl nl nl nl nl nl nl nl nl nl nl nl nl nl n...,guests,"[guest_id, gender_code, guest_first_name, gues...","[gender_code, date_of_birth]",<N> What are the number of dates of birth of a...
15781,2914@x_name@DESC,swimming,Bar,Easy,"Visualize BAR SELECT name , ID FROM swimmer OR...","Plot id by grouped by name as a bar graph , sh...",mark bar data swimmer encoding x name y aggreg...,"[id, name, time]",NaN,mark [T] data swimmer encoding x [X] y aggrega...,<N> Plot id by grouped by name as a bar graph ...,mark bar data swimmer encoding x name y aggreg...,nl 

### Spider

- The purpose of this check is to determine if the db_id, tables and mentioned column names are present in the spider database
- After checking, it was found that there are some differences. Most of the times names of tables and columns are similar but not exact making them not valid

In [25]:
spider_tables = json_to_dict(join(SPIDER_DATABASE_DIRECTORY, "tables.json"))
len(spider_tables)

166

In [26]:
spider_tables[0].keys()

dict_keys(['column_names', 'column_names_original', 'column_types', 'db_id', 'foreign_keys', 'primary_keys', 'table_names', 'table_names_original'])

In [27]:
db_ids = []
table_names = []
column_names = []

for db_info in spider_tables:
    cn = pd.DataFrame(columns=["id", "column_name"], data=db_info["column_names"])

    for i, t in enumerate(db_info["table_names"]):
        db_ids.append(db_info["db_id"])
        table_names.append(t.replace(" ", "_"))
        column_names.append(
            cn.loc[cn["id"] == i]["column_name"].str.replace(" ", "_").to_list()
        )

df_spider = pd.DataFrame()
df_spider["db_id"] = db_ids
df_spider["table"] = table_names
df_spider["columns"] = column_names

df_spider

,db_id,table,columns
0,perpetrator,perpetrator,"[perpetrator_id, people_id, date, year, locati..."
1,perpetrator,people,"[people_id, name, height, weight, home_town]"
2,college_2,classroom,"[building, room_number, capacity]"
3,college_2,department,"[department_name, building, budget]"
4,college_2,course,"[course_id, title, department_name, credits]"
...,...,...,...
871,product_catalog,attribute_definitions,"[attribute_id, attribute_name, attribute_data_..."
872,product_catalog,catalogs,"[catalog_id, catalog_name, catalog_publisher, ..."
873,product_catalog,catalog_structure,"[catalog_level_number, catalog_id, catalog_lev..."
874,product_catalog,catalog_contents,"[catalog_entry_id, catalog_level_number, paren..."


In [28]:
# Check if all the db_ids are in the master repo (spider in this case)
set(df_ncNet["db_id"].values).issubset(set(df_spider["db_id"].values))

True

In [ ]:
df = df_spider.copy(deep=True)
df.columns = ["db_id", "table_used", "spider_column"]

df = pd.merge(df_ncNet, df, on=["db_id", "table_used"], how="left")
df.loc[df["spider_column"].isna()]